In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-set/train_age_dataset.csv
/kaggle/input/test-set/test_age_dataset.csv


In [2]:
df=pd.read_csv('/kaggle/input/training-set/train_age_dataset.csv')
df.head(10)

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,...,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,265153,48958844,2,1,0.0,0.0,0.0,0,0.0,0.0,...,0.20000,0.0,0.041667,0.025000,0.0,0.000000,0.175,0.033333,0.0,1
1,405231,51100441,2,2,0.0,0.0,0.0,0,0.0,0.0,...,0.09322,0.0,0.012712,0.018644,0.0,0.084746,0.000,0.033898,82.5,2


In [3]:
df.drop(['Unnamed: 0','userId'],axis=1,inplace=True)

In [4]:
df['age_group'].value_counts()

1    308315
4     60803
3     60404
2     59355
Name: age_group, dtype: int64

In [5]:
X=df.drop(['age_group'],axis=1)
y=df['age_group']

In [6]:
import sklearn
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [7]:
from imblearn.combine import SMOTETomek

In [8]:
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_sample(X,y)

In [9]:
train_x,test_x,train_y,test_y=train_test_split(X_res,y_res,test_size=.0625,random_state=42)

In [13]:
# model=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
#               importance_type='gain', interaction_constraints='',
#               learning_rate=0.300000012, max_delta_step=0, max_depth=6,
#               min_child_weight=1,
#               n_estimators=100, n_jobs=-1, num_parallel_tree=1,
#               objective='multi:softprob', random_state=4500, reg_alpha=0,
#               reg_lambda=1, scale_pos_weight=None, subsample=1,
#               tree_method='gpu_hist',validate_parameters=1)

In [21]:
xgb_clf = XGBClassifier(tree_method = 'gpu_hist', gpu_id=0, objective = "multi:softmax", num_class=4,n_estimators=2000, max_depth=15,learning_rate=0.005,reg_lambda=3)

In [22]:
xgb_model = xgb_clf.fit(train_x, train_y)

In [23]:
xgb_model

XGBClassifier(gpu_id=0, learning_rate=0.005, max_depth=15, n_estimators=2000,
              num_class=4, objective='multi:softprob', reg_lambda=3,
              tree_method='gpu_hist')

In [ ]:
xgb_clf.feature_importances_

In [24]:
xgb_model.score(train_x,train_y)

0.8714312027997789

In [25]:
xgb_model.score(test_x,test_y)

0.7375996631844854

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
nm = NearMiss()
X_res,y_res=nm.fit_sample(X,y)

In [ ]:
X_res

In [7]:
from sklearn.preprocessing import StandardScaler
features = ['tier', 'gender', 'following_rate', 'followers_avg_age',
       'following_avg_age', 'max_repetitive_punc',
       'num_of_hashtags_per_action', 'emoji_count_per_action',
       'punctuations_per_action', 'number_of_words_per_action',
       'avgCompletion', 'avgTimeSpent', 'avgDuration', 'avgComments',
       'creations', 'content_views', 'num_of_comments',
       'weekends_trails_watched_per_day', 'weekdays_trails_watched_per_day',
       'slot1_trails_watched_per_day', 'slot2_trails_watched_per_day',
       'slot3_trails_watched_per_day', 'slot4_trails_watched_per_day', 'avgt2']
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['age_group']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=17)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10', 'f11', 'f12','f13', 'f14', 'f15','f16', 'f17'])
finalDf = pd.concat([principalDf, df[['age_group']]], axis = 1)

In [ ]:
X=finalDf.drop(['age_group'],axis=1)
y=finalDf['age_group']

In [ ]:
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=.0625,random_state=42)

In [ ]:
xgb_model.score(train_x,train_y)

In [ ]:
xgb_model.score(test_x,test_y)

In [26]:
df1=pd.read_csv('../input/test-set/test_age_dataset.csv')

In [ ]:
df1.head(2)

In [27]:
df1.drop(['Unnamed: 0','userId'],axis=1,inplace=True)

In [28]:
pred1=xgb_model.predict(df1)

In [ ]:
pred1

In [29]:
column_values = ['prediction'] 
test_df = pd.DataFrame(data = pred1,columns = column_values) 
test_df.head(2)

,prediction
0,1
1,1


In [ ]:
test_df.reset_index(drop=True, inplace=True)

In [30]:
test_df.to_csv('k5.csv', index=False)